In [3]:
from pathlib import Path
from collections import defaultdict
import pandas as pd

# === 경로 설정 ===
folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_data",
]

# 이미지 확장자
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# === 폴더별 이미지 갯수 세기 ===
counts = []
for f in folders:
    p = Path(f)
    imgs = [x for x in p.rglob("*") if x.suffix in IMG_EXTS]
    counts.append({"folder": p.name, "image_count": len(imgs)})

# DataFrame으로 변환 후 표 출력
df = pd.DataFrame(counts)
df

,folder,image_count
0,20250721_data,3727
1,20250725_data,2080
2,20250904_data,7330


In [3]:
from pathlib import Path
from collections import Counter
import pandas as pd

# === 대상 폴더 ===
folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data",
]

# === YOLO 클래스 이름 (고정 매핑) ===
CLASS_NAMES = {
    0: "Divot",
    1: "Fixed_Divot",
    2: "Diseased_Grass",
    3: "Confused_Object",
    4: "Pole",
    5: "Sprinkler",
    6: "Drain",
    7: "Golf ball",
}

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

rows = []

for f in folders:
    base = Path(f)

    # 1) 이미지 파일 수
    img_files = [p for p in base.rglob("*") if p.suffix in IMG_EXTS]
    img_count = len(img_files)

    # 2) classes.txt 제외 라벨 파일
    label_files = [p for p in base.rglob("*.txt") if p.name != "classes.txt"]

    # 3) 클래스별 객체 수 (레이블 라인 안전 파싱)
    counter = Counter()
    for txt in label_files:
        with open(txt, "r", encoding="utf-8") as lf:
            for line in lf:
                s = line.strip()
                if not s:
                    continue
                parts = s.split()
                try:
                    cid = int(parts[0])
                except Exception:
                    continue  # 첫 토큰이 정수가 아니면 스킵
                counter[cid] += 1

    # 4) 결과 행
    row = {"folder": base.name, "image_count": img_count}
    for cid, cname in CLASS_NAMES.items():
        row[cname] = counter.get(cid, 0)
    rows.append(row)

# === DataFrame ===
df = pd.DataFrame(rows)

# 숫자 열만 0으로 채우고 정수형으로 변환
numeric_cols = [c for c in df.columns if c != "folder"]
df[numeric_cols] = df[numeric_cols].fillna(0).apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)

# 보기 좋게 정렬(폴더명 인덱스)
df = df.set_index("folder")

# 열 순서: image_count + 클래스 순서
df = df[["image_count"] + list(CLASS_NAMES.values())]

df

,image_count,Divot,Fixed_Divot,Diseased_Grass,Confused_Object,Pole,Sprinkler,Drain,Golf ball
folder,,,,,,,,,
20250721_good_data,350,376,309,12,3,0,0,0,0
20250725_good_data,169,110,132,0,0,12,24,7,13
20250904_good_data,3625,3856,4720,545,0,106,40,261,157


In [4]:
df_wide = df.transpose()

df_wide

folder,20250721_good_data,20250725_good_data,20250904_good_data
image_count,350,169,3625
Divot,376,110,3856
Fixed_Divot,309,132,4720
Diseased_Grass,12,0,545
Confused_Object,3,0,0
Pole,0,12,106
Sprinkler,0,24,40
Drain,0,7,261
Golf ball,0,13,157


In [11]:
from pathlib import Path
import shutil

# ======= 원본/목적지 경로 =======
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data")
DST = SRC.parent / "20250725_bad_data_yolo"

IMAGES_DIR = DST / "images"
LABELS_DIR = DST / "labels"

# ======= 폴더 생성 =======
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
LABELS_DIR.mkdir(parents=True, exist_ok=True)

# 이미지와 라벨 확장자 정의
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
LABEL_EXT = ".txt"

# ======= 파일 복사 =======
for p in SRC.iterdir():
    if p.is_file():
        suffix = p.suffix.lower()
        if suffix in IMG_EXTS:
            shutil.copy2(p, IMAGES_DIR / p.name)   # 이미지 복사
        elif suffix == LABEL_EXT:
            shutil.copy2(p, LABELS_DIR / p.name)   # 라벨 복사

print("복사 및 분류 완료!")
print(f"이미지 → {IMAGES_DIR}")
print(f"라벨  → {LABELS_DIR}")

복사 및 분류 완료!
이미지 → /home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data_yolo/images
라벨  → /home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data_yolo/labels


In [19]:
from pathlib import Path

folder = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data")

# 확장자별 이름(확장자 제외) 집합
jpg_names = {p.stem for p in folder.glob("*.jpg")}
jpg_names |= {p.stem for p in folder.glob("*.jpeg")}
txt_names = {p.stem for p in folder.glob("*.txt")}

# 집합 연산
only_jpg  = jpg_names - txt_names      # jpg만 있고 txt 없는 파일
only_txt  = txt_names - jpg_names      # txt만 있고 jpg 없는 파일
both      = jpg_names & txt_names      # 둘 다 있는 파일

print(f"JPG만 있는 파일 개수 : {len(only_jpg)}")
print(f"TXT만 있는 파일 개수 : {len(only_txt)}")
print(f"둘 다 있는 파일 개수 : {len(both)}")

# 필요하면 이름까지 보고 싶다면 주석 해제

print("JPG only:", sorted(only_jpg))
#print("TXT only:", sorted(only_txt))
# print("Both    :", sorted(both))

JPG만 있는 파일 개수 : 0
TXT만 있는 파일 개수 : 1
둘 다 있는 파일 개수 : 386
JPG only: []


In [20]:
from pathlib import Path

bad_dir  = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data")
good_dir = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data")

# 확장자 제거한 파일 이름 집합 만들기
def base_names(folder, exts):
    return {p.stem for p in folder.glob("*") if p.suffix.lower() in exts}

# 각 확장자별
jpg_bad  = base_names(bad_dir,  {".jpg", ".jpeg"})
txt_bad  = base_names(bad_dir,  {".txt"})
jpg_good = base_names(good_dir, {".jpg", ".jpeg"})
txt_good = base_names(good_dir, {".txt"})

# 교집합 계산
jpg_overlap = jpg_bad & jpg_good
txt_overlap = txt_bad & txt_good

print(f"JPG 겹치는 파일 개수 : {len(jpg_overlap)}")
print(f"TXT 겹치는 파일 개수 : {len(txt_overlap)}")

# 필요하면 이름 리스트 출력
print("JPG overlap names:", sorted(jpg_overlap))
print("TXT overlap names:", sorted(txt_overlap))


JPG 겹치는 파일 개수 : 0
TXT 겹치는 파일 개수 : 1
JPG overlap names: []
TXT overlap names: ['classes']


In [14]:
from pathlib import Path

good_dir = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")
bad_dir  = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data")

# 이름(확장자 제외) 집합 만들기
def base_names(folder, exts):
    return {p.stem for p in folder.glob("*") if p.suffix.lower() in exts}

good_names = base_names(good_dir, {".jpg", ".jpeg", ".txt"})
bad_files  = [p for p in bad_dir.glob("*") if p.suffix.lower() in {".jpg", ".jpeg", ".txt"}]

# good과 이름이 겹치는 bad 파일 삭제
deleted = []
for f in bad_files:
    if f.stem in good_names:
        f.unlink()  # 실제 삭제
        deleted.append(f)

print(f"삭제된 파일 수 : {len(deleted)}")
for p in deleted:
    print("삭제:", p)

삭제된 파일 수 : 7171
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_00949.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_02206.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_04337.txt
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_04606.txt
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_02524.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_03987.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_04844.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_02034.txt
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_00364.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data/20250904_01941.jpg
삭제: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_